In [1]:
// Import required packages 

import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer

Intitializing Scala interpreter ...

Spark Web UI available at http://8c3c88d7baee:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1590318209499)
SparkSession available as 'spark'


import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer


In [2]:
// Read the csv file into a dataframe

val sqlContext = new SQLContext(sc)

val df = sqlContext
    .read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("weatherAUS.csv")

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@68d2ac0d
df: org.apache.spark.sql.DataFrame = [Date: string, Location: string ... 25 more fields]


In [3]:
// Cast columns as appropriate types

val df2 = df.withColumn("Date",to_date($"Date","dd/MM/yyyy"))
            .withColumn("MinTemp",col("MinTemp").cast(DoubleType))
            .withColumn("MaxTemp",col("MaxTemp").cast(DoubleType))
            .withColumn("Rainfall",col("Rainfall").cast(DoubleType))
            .withColumn("Evaporation",col("Evaporation").cast(DoubleType))
            .withColumn("Sunshine",col("Sunshine").cast(DoubleType))
            .withColumn("WindGustSpeed",col("WindGustSpeed").cast(DoubleType))
            .withColumn("WindSpeed9am",col("WindSpeed9am").cast(DoubleType))
            .withColumn("WindSpeed3pm",col("WindSpeed3pm").cast(DoubleType))
            .withColumn("Humidity9am",col("Humidity9am").cast(DoubleType))
            .withColumn("Humidity3pm",col("Humidity3pm").cast(DoubleType))
            .withColumn("Pressure9am",col("Pressure9am").cast(DoubleType))
            .withColumn("Pressure3pm",col("Pressure3pm").cast(DoubleType))
            .withColumn("Cloud9am",col("Cloud9am").cast(DoubleType))
            .withColumn("Cloud3pm",col("Cloud3pm").cast(DoubleType))
            .withColumn("Temp9am",col("Temp9am").cast(DoubleType))
            .withColumn("Temp3pm",col("Temp3pm").cast(DoubleType))
df2.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Pressure3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- _

df2: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 25 more fields]


In [4]:
// Remove any values from numeric columns which are outside expected range

val df3 = df2.withColumn("MinTemp",when(col("MinTemp")<(-10),null).otherwise(col("MinTemp")))
            .withColumn("MinTemp",when(col("MinTemp")>(50),null).otherwise(col("MinTemp")))
            .withColumn("MaxTemp",when(col("MaxTemp")<(-10),null).otherwise(col("MaxTemp")))
            .withColumn("MaxTemp",when(col("MaxTemp")>(50),null).otherwise(col("MaxTemp")))
            .withColumn("Temp9am",when(col("Temp9am")<(-10),null).otherwise(col("Temp9am")))
            .withColumn("Temp9am",when(col("Temp9am")>(50),null).otherwise(col("Temp9am")))
            .withColumn("Temp3pm",when(col("Temp3pm")<(-10),null).otherwise(col("Temp3pm")))
            .withColumn("Temp3pm",when(col("Temp3pm")>(50),null).otherwise(col("Temp3pm")))
            .withColumn("Cloud9am",when(col("Cloud9am")<(0),null).otherwise(col("Cloud9am")))
            .withColumn("Cloud9am",when(col("Cloud9am")>(9),null).otherwise(col("Cloud9am")))
            .withColumn("Cloud3pm",when(col("Cloud3pm")<(0),null).otherwise(col("Cloud3pm")))
            .withColumn("Cloud3pm",when(col("Cloud3pm")>(9),null).otherwise(col("Cloud3pm")))
            .withColumn("Humidity9am",when(col("Humidity9am")<(0),null).otherwise(col("Humidity9am")))
            .withColumn("Humidity9am",when(col("Humidity9am")>(100),null).otherwise(col("Humidity9am")))
            .withColumn("Humidity3pm",when(col("Humidity3pm")<(0),null).otherwise(col("Humidity3pm")))
            .withColumn("Humidity3pm",when(col("Humidity3pm")>(100),null).otherwise(col("Humidity3pm")))
            .withColumn("Pressure9am",when(col("Pressure9am")<(970),null).otherwise(col("Pressure9am")))
            .withColumn("Pressure9am",when(col("Pressure9am")>(1050),null).otherwise(col("Pressure9am")))
            .withColumn("Pressure3pm",when(col("Pressure3pm")<(970),null).otherwise(col("Pressure3pm")))
            .withColumn("Pressure3pm",when(col("Pressure3pm")>(1050),null).otherwise(col("Pressure3pm")))
            .withColumn("WindGustSpeed",when(col("WindGustSpeed")<(0),null).otherwise(col("WindGustSpeed")))
            .withColumn("WindGustSpeed",when(col("WindGustSpeed")>(150),null).otherwise(col("WindGustSpeed")))
            .withColumn("WindSpeed9am",when(col("WindSpeed9am")<(0),null).otherwise(col("WindSpeed9am")))
            .withColumn("WindSpeed9am",when(col("WindSpeed9am")>(150),null).otherwise(col("WindSpeed9am")))
            .withColumn("WindSpeed3pm",when(col("WindSpeed3pm")<(0),null).otherwise(col("WindSpeed3pm")))
            .withColumn("WindSpeed3pm",when(col("WindSpeed3pm")>(150),null).otherwise(col("WindSpeed3pm")))
            .withColumn("Rainfall",when(col("Rainfall")<(0),null).otherwise(col("Rainfall")))
            .withColumn("Rainfall",when(col("Rainfall")>(500),null).otherwise(col("Rainfall")))
            .withColumn("Evaporation",when(col("Evaporation")<(0),null).otherwise(col("Evaporation")))
            .withColumn("Evaporation",when(col("Evaporation")>(100),null).otherwise(col("Evaporation")))
            .withColumn("Sunshine",when(col("Sunshine")<(0),null).otherwise(col("Sunshine")))
            .withColumn("Sunshine",when(col("Sunshine")>(15),null).otherwise(col("Sunshine")))

df3: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 25 more fields]


In [5]:
// Extract the month from the "Date" column and insert it as a new column

val df4 = df3.withColumn("Month",month(col("Date")))

df4: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 26 more fields]


In [6]:
// Remove duplicates based on Date and Location

val df5 = df4.dropDuplicates("Date","Location")
println("Rows dropped = "+(df3.count()-df4.count()))

2020-05-24 11:03:47,493 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Rows dropped = 0


df5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: date, Location: string ... 26 more fields]


In [7]:
// Replace null values with average for each numeric column

val NumCols = Array("MinTemp","MaxTemp","Temp9am","Temp3pm","Pressure9am","Pressure3pm","WindGustSpeed","WindSpeed9am",
                    "WindSpeed3pm","Rainfall","Evaporation","Sunshine","Cloud9am","Cloud3pm","Humidity9am","Humidity3pm")

val imputer = new Imputer()
  .setInputCols(NumCols)
  .setOutputCols(NumCols)
  .setStrategy("mean")

val df6 = imputer.fit(df5).transform(df5)

NumCols: Array[String] = Array(MinTemp, MaxTemp, Temp9am, Temp3pm, Pressure9am, Pressure3pm, WindGustSpeed, WindSpeed9am, WindSpeed3pm, Rainfall, Evaporation, Sunshine, Cloud9am, Cloud3pm, Humidity9am, Humidity3pm)
imputer: org.apache.spark.ml.feature.Imputer = imputer_0bf742de7dd1
df6: org.apache.spark.sql.DataFrame = [Date: date, Location: string ... 26 more fields]


In [8]:
// Create a list of unique locations, for potential use to improve imputations

val location_array = df5.select("Location").distinct().rdd.map(r => r(0)).collect()
val location_list = location_array.toList
location_list

location_array: Array[Any] = Array(Cairns, NorfolkIsland, Bendigo, Walpole, Canberra, Woomera, Adelaide, Cobar, SydneyAirport, PerthAirport, Wollongong, Williamtown, Moree, Mildura, Portland, Albany, SalmonGums, Brisbane, Sydney, Perth, Sale, BadgerysCreek, Hobart, Tuggeranong, Ballarat, GoldCoast, MelbourneAirport, Dartmoor, Nhil, PearceRAAF, Albury, Witchcliffe, WaggaWagga, Darwin, Uluru, Nuriootpa, CoffsHarbour, Melbourne, Penrith, MountGambier, NorahHead, Katherine, MountGinini, Townsville, Newcastle, AliceSprings, Watsonia, Richmond, Launceston)
location_list: List[Any] = List(Cairns, NorfolkIsland, Bendigo, Walpole, Canberra, Woomera, Adelaide, Cobar, SydneyAirport, PerthAirport, Wollongong, Williamtown, Moree, Mildura, Portland, Albany, SalmonGums, Brisbane, Sydney, Perth, Sale, ...

### Note: This section of code does not work, I was trying to split the dataframe up by location in order to improve the imputations

In [9]:
// val byLocArray = location_array.map(location => df5.where($"Location" <=> location))

In [10]:
// val dfArray = df5.where(df5.col("Location") == x) for (x <- location_list)

In [11]:
// for (x <- location_list){
//    val dfArray = (df5.filter(df5("Location") === x))
//}

## Various ways to display, filter and aggregate the data

In [12]:
df6.groupBy("Pressure3pm").count().sort(desc("count")).show()

+------------------+-----+
|       Pressure3pm|count|
+------------------+-----+
|1015.2582035378904|13981|
|            1015.5|  773|
|            1015.3|  767|
|            1015.7|  763|
|            1015.6|  761|
|            1015.1|  752|
|            1015.8|  751|
|            1013.5|  751|
|            1015.4|  745|
|            1016.0|  738|
|            1014.8|  735|
|            1015.2|  730|
|            1014.9|  727|
|            1013.4|  723|
|            1016.5|  719|
|            1013.3|  717|
|            1013.7|  717|
|            1014.2|  716|
|            1017.1|  715|
|            1013.9|  711|
+------------------+-----+
only showing top 20 rows



In [13]:
df6.groupBy("WindGustDir").count().show()

+-----------+-----+
|WindGustDir|count|
+-----------+-----+
|        SSE| 8993|
|         NW| 8003|
|         SW| 8797|
|         NA| 9330|
|          E| 9071|
|        WSW| 8901|
|        ENE| 7992|
|         NE| 7060|
|        NNW| 6561|
|          N| 9033|
|        SSW| 8610|
|          W| 9780|
|          S| 8949|
|         SE| 9309|
|        WNW| 8066|
|        NNE| 6433|
|        ESE| 7305|
+-----------+-----+



In [14]:
df6.filter("Location == 'Albury'").select("Date","MinTemp","MaxTemp","Rainfall","Evaporation","Sunshine").show(20)

+----------+-------+-------+--------+-----------------+-----------------+
|      Date|MinTemp|MaxTemp|Rainfall|      Evaporation|         Sunshine|
+----------+-------+-------+--------+-----------------+-----------------+
|2010-07-26|    1.9|   14.9|     0.2|5.468109011788718|7.624853113193593|
|2011-05-02|   12.3|   22.3|     0.0|5.468109011788718|7.624853113193593|
|2011-05-28|    2.6|   13.9|     0.0|5.468109011788718|7.624853113193593|
|2011-10-24|   18.0|   28.4|     0.0|5.468109011788718|7.624853113193593|
|2011-12-22|   16.4|   29.6|     0.6|5.468109011788718|7.624853113193593|
|2013-07-28|    3.0|   16.1|     0.2|5.468109011788718|7.624853113193593|
|2013-09-03|    8.0|   23.3|     0.0|5.468109011788718|7.624853113193593|
|2014-02-09|   18.8|   42.0|     0.0|5.468109011788718|7.624853113193593|
|2014-07-11|    7.1|   11.3|     0.4|5.468109011788718|7.624853113193593|
|2015-02-04|   15.2|   29.5|     0.0|5.468109011788718|7.624853113193593|
|2015-08-04|   -2.4|    8.6|     1.4|5

In [15]:
df6.select("MinTemp","MaxTemp","Temp9am","Temp3pm","Pressure9am","Pressure3pm").describe().show()
df6.select("WindGustSpeed","WindSpeed9am","WindSpeed3pm","Rainfall","Evaporation","Sunshine").describe().show()
df6.select("Cloud9am","Cloud3pm","Humidity9am","Humidity3pm").describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+
|summary|           MinTemp|          MaxTemp|           Temp9am|           Temp3pm|       Pressure9am|       Pressure3pm|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+
|  count|            142193|           142193|            142193|            142193|            142193|            142193|
|   mean|12.186399728729262|23.22678419127236|16.987508581701345|21.687234973147774|1017.6537584159646|1015.2582035378907|
| stddev|6.3889236757881696|7.109554494595336| 6.472165978773238| 6.870770835439069| 6.746248325436812| 6.681787586654765|
|    min|              -8.5|             -4.8|              -7.2|              -5.4|             980.5|             977.1|
|    max|              33.9|             48.1|              40.2|              46.7|            1041.0|            1039.6|
+-------+-------

In [16]:
df5.groupBy("Location").agg(max("Rainfall")).show()

+-------------+-------------+
|     Location|max(Rainfall)|
+-------------+-------------+
|       Cairns|        278.4|
|NorfolkIsland|        156.8|
|      Bendigo|         66.4|
|      Walpole|         93.2|
|     Canberra|         87.0|
|      Woomera|         66.4|
|     Adelaide|         75.2|
|        Cobar|         68.0|
|SydneyAirport|        106.8|
| PerthAirport|         76.6|
|   Wollongong|        192.0|
|  Williamtown|        225.0|
|        Moree|        113.0|
|      Mildura|        155.0|
|     Portland|         72.0|
|       Albany|         49.4|
|   SalmonGums|         55.4|
|     Brisbane|        182.6|
|       Sydney|        119.4|
|        Perth|        114.4|
+-------------+-------------+
only showing top 20 rows

